# Public Kernals Review
https://www.kaggle.com/c/pubg-finish-placement-prediction/kernels

## EDA is FUN!
https://www.kaggle.com/deffro/eda-is-fun

```python
df_features = pd.DataFrame()
df_features['players_joined'] = x.groupby('match_id')['match_id'].transform('count')
df_features['total_distance'] = x['ride_distance'] + x['walk_distance'] + x['swim_distance']
df_features['kills_norm'] = x['kills'] * ((100 - df_features['players_joined']) / 100 + 1)
df_features['damage_dealt_norm'] = x['damage_dealt'] * ((100 - df_features['players_joined']) / 100 + 1)
df_features['heals_and_boosts'] = x['heals'] + x['boosts']
df_features['total_distance'] = x['walk_distance'] + x['ride_distance'] + x['swim_distance']
df_features['boosts_per_walk_distance'] = x['boosts'] / (x['walk_distance'] + 1)
df_features['boosts_per_walk_distance'].fillna(0, inplace=True)
df_features['heals_per_walk_distance'] = x['heals'] / (x['walk_distance'] + 1)
df_features['heals_per_walk_distance'].fillna(0, inplace=True)
df_features['heals_and_boosts_per_walk_distance'] = df_features['heals_and_boosts'] / (x['walk_distance'] + 1)
df_features['heals_and_boosts_per_walk_distance'].fillna(0, inplace=True)
df_features['kills_per_walk_distance'] = x['kills'] / (x['walk_distance'] + 1)
df_features['kills_per_walk_distance'].fillna(0, inplace=True)
df_features['team'] = [1 if i > 50 else 2 if (bool(i > 25) & bool(i <= 50)) else 4 for i in x['num_groups']]
```

## Simple NN Baseline 4
https://www.kaggle.com/anycode/simple-nn-baseline-4 <br>
Public Score: 0.0244 <br>
Run Time: 12600s = 3h 30m <br>

Усредненный таргет для групп. TODO: зачем потом делается y = 2*y - 1?
```python
y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
```


```python
print("get group mean feature")
agg = df.groupby(['matchId','groupId'])[features].agg('mean')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
```

```python
# print("get group sum feature")
# agg = df.groupby(['matchId','groupId'])[features].agg('sum')
# agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
```

```python
# print("get group sum feature")
# agg = df.groupby(['matchId','groupId'])[features].agg('sum')
# agg_rank = agg.groupby('matchId')[features].agg('sum')
```

Максимальное значение показателя в пати
```python
print("get group max feature")
agg = df.groupby(['matchId','groupId'])[features].agg('max')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
```

Минимально значение показателя в пати
```python
print("get group min feature")
agg = df.groupby(['matchId','groupId'])[features].agg('min')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
```

Количество человек в пати
```python
print("get group size feature")
agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
```

Среднее значение показателя в матчеч
```python
print("get match mean feature")
agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
```

```python
# print("get match type feature")
# agg = df.groupby(['matchId'])[matchType.columns].agg('mean').reset_index()
```

Количество участников в матче, проверить совпадает ли с numGroups
```python
print("get match size feature")
agg = df.groupby(['matchId']).size().reset_index(name='match_size')
```

```python
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=False).fit(x_train)
```

Таргет приводится к шкале [-1, 1], после предсказкния его нужно привести обратно к [0, 1], причем стоит округлять до 2 знаков после запятой, потому что таргет не простое вещественное число, а позиция от 0 до 100.
```python
y = y * 2 - 1
```

NN с 3 Dense слоями. Тут используется a2m2l активация - хз че это. На форуме советуют попробать (leaky) ReLU. <br>
RunTime: about 4 hours. running 2 hours can get a score of about 0.025
```python
epoch_decay = 2
epoch_train = epoch_decay * 18
rate_init=0.08
hidden_size = 32
verbose=1
activation='a2m2l'
leaky = -0.001

mlp = MLP(layer_size=[x_train.shape[1], hidden_size, hidden_size, hidden_size, 1],activation=activation, leaky=leaky,bias_rate=[], regularization=1,importance_mul=0.0001, output_shrink=0.1, output_range=[-1,1], loss_type="hardmse")
feature_importance = mlp.train(x_train, y, verbose=verbose, importance_out=True, iteration_log=20000, rate_init=rate_init, rate_decay=0.8, epoch_train=epoch_train, epoch_decay=epoch_decay)
```

Ограничение значений от 0 до 1
```python
np.clip(x_test, out=x_test, a_min=-1, a_max=1)
```

Дополнительная корректировака предсказаний<br>
Can you explain what the "fix winPlacePerc" section does? Also I don't think I understand what the maxPlace column signifies.
<br><br>
fix winPlacePerc will contribute about 50% better, the target = final_place / maxplace, so the target is discrete, not continuous values.
```python
print("fix winPlacePerc")
for i in range(len(df_test)):
    winPlacePerc = pred[i]
    maxPlace = int(df_test.iloc[i]['maxPlace'])
    if maxPlace == 0:
        winPlacePerc = 0.0
    elif maxPlace == 1:
        winPlacePerc = 1.0
    else:
        gap = 1.0 / (maxPlace - 1)
        winPlacePerc = round(winPlacePerc / gap) * gap
    
    if winPlacePerc < 0: winPlacePerc = 0.0
    if winPlacePerc > 1: winPlacePerc = 1.0    
    pred[i] = winPlacePerc

    if (i + 1) % 100000 == 0:
        print(i, flush=True, end=" ")

df_test['winPlacePerc'] = pred
```

Идеи в обсуждениях:

I think sum of features within same group / sum of that within same match will work better

sumOfKillsPerGroup / sumOfKillsPerMatch

that is percent kills of each group, the higher, the better, and is better than percent rank feature
<br>
<br>
<br>
the "fix winPlacePerc" section has some bugs, because I use round(winPlacePerc / gap) * gap, so there may have two team having same winPlacePerc which is not possible

Also numGroups is different from matchsize, I do not know why df.groupby(['matchId']).size().resetindex(name='match_size')

DBNOs can also be used, because if we know DBNOs the enemy players, that means the mean feature of group should be fixed, the dataset do not give all the data of players within the same group

TODO: https://www.kaggle.com/pecooper/leaky-chicken-dinner/code
Советовал автор Simple NN Baseline 4


# How to Score (0.0255 - 0.0245)
Разбор Simple NN Baseline 4
https://www.kaggle.com/modmari/how-to-score-0-0255-0-0245-top-10-score

# Effective feature engineering
https://www.kaggle.com/rejasupotaro/effective-feature-engineering

- Correlation
- Score gain on a simple model
- Feature importances of Tree models
- Permutation importance
- SHAP values
- Score gain on a complex model

# PUBG LGB Baseline (LB ~0.0349)
Под капотом LightGBM<br>
Public Score: 0.0349<br>
Run Time: 6730 seconds = 2h<br>

# A Simple Post-Processing Trick (LB .0237 -> .0204)

https://www.kaggle.com/ceshine/a-simple-post-processing-trick-lb-0237-0204


https://www.kaggle.com/chocozzz/pubg-data-description-a-to-z-fe-with-python